### [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()
- 세션ID 설정하고 대화를 진행합니다(multi-turn conversation)
- LLM 모델이 과거 대화를 기억하지 못하는 상황을 만드세요. 

In [20]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages
from operator import itemgetter
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_completion_tokens=500
    )
messages = [
	  ('system','''당신은 이력서 작성 컨설턴트입니다. \n 아래 정보를 바탕으로 지원자 입장에서 1200자 이내로 이력서를 작성합니다.
    문장은 자연스럽고 매끄럽게 작성합니다.'''),
	  ('placeholder','{chat_history}'),
	  ('user','{query}')
    ]

In [22]:
prompt = ChatPromptTemplate.from_messages(messages)

trimmer = trim_messages(
    max_token = 70,
    strategy = 'last',
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = 'human',    
)

chain =( RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
        | prompt
        | model)

chain

RunnableAssign(mapper={
  chat_history: RunnableLambda(itemgetter('chat_history'))
                | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['query'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, T

In [18]:
store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


wr_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    history_messages_key='chat_history',
    input_messages_key='query'
)

while True:
    query = input('질문을 입력하세요. 종료시 s입력')
    if query.upper() == 'S':
        break

    wr_chain.invoke(
        {'query': query },
        config={'configurable':{'session_id':'ywgw'}}
    )
    print(f'\n {'='*50}')
    

[이력서]

지원자: 솔  
경력: 신입  
연락처: [휴대전화번호]  
이메일: [이메일 주소]  

개인소개서  
저는 신입으로서 파이썬 프로그래밍과 최근 주목받는 대형 언어 모델(LLM) 및 랭체인(LangChain) 기술에 관심을 가지고 있습니다. 데이터를 다루고 분석하는 능력을 갖추었으며, 빠르게 변화하는 인공지능 분야에서 실무 역량을 쌓고자 열정을 가지고 있습니다. 문제 해결과 창의적 사고를 바탕으로 새로운 기술을 빠르게 습득하려 노력하며, 팀과 협력하여 목표를 달성하는 데에 적극적입니다.

기술 역량  
- 프로그래밍: 파이썬(기본 적 이해 및 활용 가능)  
- 인공지능: 대형 언어 모델(LLM) 활용 및 이해  
- 프레임워크 및 라이브러리: 랭체인(LangChain) 활용 경험

학력 및 경력  
- [학력 사항 입력: 학교명, 전공, 졸업 연도]  
- 인턴 또는 프로젝트 경험이 있다면 간략히 기재 가능  

자기개발 및 프로젝트 경험  
저는 파이썬을 활용하여 간단한 데이터 분석 및 자동화 프로젝트 경험이 있으며, 온라인 강의 또는 스터디를 통해 LLM과 랭체인 기술에 대한 실습과 이해를 높여 왔습니다. 이를 통해 자연어 처리 및 AI 솔루션 개발에 대한 관심과 기본 역량을 쌓았습니다.

포부  
이번 기회를 통해 실제 업무 환경에서 최신 AI 기술을 적용하며 성장하고 싶습니다. 지속적인 학습과 팀워크를 통해 회사의 발전에 기여하는 인재가 되겠습니다.

감사합니다.
당장 취업을 위해 준비해야 할 공부는 다음과 같이 정리할 수 있습니다.

1. **파이썬 심화 학습**: 이미 기본 수준이지만, 실무에 바로 적용할 수 있도록 데이터 처리, 웹 크롤링, 자동화, API 활용 등 다양한 파이썬 기술을 익히는 것이 중요합니다. 특히, 데이터 분석 라이브러리(Numpy, Pandas, Matplotlib) 사용 능력을 키우세요.

2. **인공지능 및 자연어 처리(NLP) 기초와 실습**: 대형 언어 모델(LLM)에 대한 이해를 높이기 위해, 자연어

In [8]:
get_session_history('ywgw')

InMemoryChatMessageHistory(messages=[HumanMessage(content='" 이름:솔라나, 나이:30, 경력:신입, 기술:파이썬, LLM" 이 정보 바탕으로 500자 내로 이력서 작성해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='성명: 솔라나  \n나이: 30세  \n경력: 신입  \n\n저는 파이썬을 활용한 프로그래밍에 자신이 있으며, 최근 대형 언어 모델(LLM) 관련 프로젝트를 통해 자연어 처리와 인공지능 분야에 대한 폭넓은 지식을 쌓았습니다. 신입이지만, 빠른 학습 능력과 열정으로 다양한 기술을 습득하고 실무에 적용하는 데 적극적입니다. 특히 파이썬을 이용한 데이터 분석 및 모델링에 강하며, LLM을 활용한 텍스트 생성, 분류, 요약 등 프로젝트 경험을 통해 문제 해결 능력을 키웠습니다. 끊임없이 성장하고자 하는 자세로, 조직에 새로운 가치를 더하며 함께 발전하고 싶습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370'}, id='run--0d66ba2b-b33e-4941-a901-25c10f6fa6ee-0'), HumanMessage(content='내가 더 공부해야할 부분에 대해  짧게 알려줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='파이썬 기반의 딥러닝 프레임워크(예: TensorFlow, PyTorch)에 대한 이해를 높이고, 자연어 처리(NLP) 분야의 최신 연구와 기술 동향을 지속적으로 학습하는 것이 좋겠습니다. 또한, 데이터 전처리와 모델 최적화, 배포 관련 기술도 함께 공부하면 실무에 더 큰 도움이 될 것입니다.', additional_kw

In [11]:
messages_list = get_session_history('ywgw').messages

In [13]:
for m in messages_list:
    print([m.type.upper()],m.content)

['HUMAN'] " 이름:솔라나, 나이:30, 경력:신입, 기술:파이썬, LLM" 이 정보 바탕으로 500자 내로 이력서 작성해줘.
['AI'] 성명: 솔라나  
나이: 30세  
경력: 신입  

저는 파이썬을 활용한 프로그래밍에 자신이 있으며, 최근 대형 언어 모델(LLM) 관련 프로젝트를 통해 자연어 처리와 인공지능 분야에 대한 폭넓은 지식을 쌓았습니다. 신입이지만, 빠른 학습 능력과 열정으로 다양한 기술을 습득하고 실무에 적용하는 데 적극적입니다. 특히 파이썬을 이용한 데이터 분석 및 모델링에 강하며, LLM을 활용한 텍스트 생성, 분류, 요약 등 프로젝트 경험을 통해 문제 해결 능력을 키웠습니다. 끊임없이 성장하고자 하는 자세로, 조직에 새로운 가치를 더하며 함께 발전하고 싶습니다.
['HUMAN'] 내가 더 공부해야할 부분에 대해  짧게 알려줘.
['AI'] 파이썬 기반의 딥러닝 프레임워크(예: TensorFlow, PyTorch)에 대한 이해를 높이고, 자연어 처리(NLP) 분야의 최신 연구와 기술 동향을 지속적으로 학습하는 것이 좋겠습니다. 또한, 데이터 전처리와 모델 최적화, 배포 관련 기술도 함께 공부하면 실무에 더 큰 도움이 될 것입니다.


In [14]:
store

{'ywgw': InMemoryChatMessageHistory(messages=[HumanMessage(content='" 이름:솔라나, 나이:30, 경력:신입, 기술:파이썬, LLM" 이 정보 바탕으로 500자 내로 이력서 작성해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='성명: 솔라나  \n나이: 30세  \n경력: 신입  \n\n저는 파이썬을 활용한 프로그래밍에 자신이 있으며, 최근 대형 언어 모델(LLM) 관련 프로젝트를 통해 자연어 처리와 인공지능 분야에 대한 폭넓은 지식을 쌓았습니다. 신입이지만, 빠른 학습 능력과 열정으로 다양한 기술을 습득하고 실무에 적용하는 데 적극적입니다. 특히 파이썬을 이용한 데이터 분석 및 모델링에 강하며, LLM을 활용한 텍스트 생성, 분류, 요약 등 프로젝트 경험을 통해 문제 해결 능력을 키웠습니다. 끊임없이 성장하고자 하는 자세로, 조직에 새로운 가치를 더하며 함께 발전하고 싶습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370'}, id='run--0d66ba2b-b33e-4941-a901-25c10f6fa6ee-0'), HumanMessage(content='내가 더 공부해야할 부분에 대해  짧게 알려줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='파이썬 기반의 딥러닝 프레임워크(예: TensorFlow, PyTorch)에 대한 이해를 높이고, 자연어 처리(NLP) 분야의 최신 연구와 기술 동향을 지속적으로 학습하는 것이 좋겠습니다. 또한, 데이터 전처리와 모델 최적화, 배포 관련 기술도 함께 공부하면 실무에 더 큰 도움이 될 것입니다.', addi

In [15]:
len(store)

1